In [1]:
import os
import h5py
import numpy as np
from IPython.lib.deepreload import reload
import matplotlib.pyplot as plt
from collections import OrderedDict
import skimage.io as io
%matplotlib qt

In [2]:
import xrdmaptools
from xrdmaptools.XRDMap import XRDMap
from xrdmaptools.reflections.SpotModels import GaussianFunctions
reload(xrdmaptools);

c:\Users\emusterma\AppData\Local\anaconda3\envs\sXRDMap-py311-1.0\Lib\site-packages\pyopencl\cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(


Connecting to databrokers...failed.


In [16]:
from orix.crystal_map import CrystalMap
from orix.quaternion import Rotation as orix_Rotation
from orix.quaternion import Orientation as orix_Orientation
from orix.crystal_map.phase_list import Phase as orix_Phase
from orix.crystal_map.phase_list import PhaseList

In [401]:
orientation_map = np.empty((*test.map.map_shape, 3, 3))

for index in range(test.map.num_images):
    indices = np.unravel_index(index, test.map.map_shape)
    
    if np.any(np.isnan(euler_map[indices])):
        orientation_map[indices] = np.nan
    else:
        orientation_map[indices] = Rotation.from_euler('zxz', euler_map[indices], degrees=True).as_matrix()

In [35]:
o_r = orix_Rotation.from_matrix(orientation_map.reshape(-1, 3, 3))

In [36]:
phase_id = []
x_list = []
y_list = []

for index in range(test.map.num_images):
    indices = np.unravel_index(index, test.map.map_shape)

    if np.any(np.isnan(orientation_map[indices])):
        phase_id.append(-1)
    else:
        phase_id.append(0)

    x_list.append(test.pos_dict['map_x'][indices])
    y_list.append(test.pos_dict['map_y'][indices])

In [ ]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
#test.clear_phases()
#test.load_phase('AMCSD\\Stibnite_0008636.cif', wd=cif_dir, phase_name="stibnite")
##test.load_phase('Unknown\\LiNbO3.cif', wd=cif_dir, phase_name="LiNbO3 new")
#test.load_phase('Barker\\1521772COD.cif', wd=cif_dir, phase_name="liNbO3 old")
#test.load_phase('Barker\\LiNbO3.cif', wd=cif_dir, phase_name="LiNbO3 148")

orix_phase = orix_Phase.from_cif(f'{cif_dir}AMCSD\\Stibnite_0008636.cif')
orix_phase.name = 'stibnite'
orix_phase = orix_Phase.from_cif(f'{cif_dir}Unknown\\LiNbO3.cif')
#orix_phase = orix_Phase.from_cif(f'{cif_dir}Barker\\1521772COD.cif')
#orix_phase = orix_Phase.from_cif(f'{cif_dir}Barker\\LiNbO3.cif')
orix_phase.name = 'linbo3'

In [63]:
x = np.linspace(*test.map_extent()[:2], test.map.map_shape[1])
y = np.linspace(*test.map_extent()[2:], test.map.map_shape[0])

#x = range(test.map.map_shape[1])
#y = range(test.map.map_shape[0])

xx, yy = np.meshgrid(x, y)
xx -= np.min(xx)
yy -= np.min(yy)

crystalmap = CrystalMap(rotations=o_r, phase_id=np.array(phase_id), x=xx.ravel(), y=yy.ravel(), phase_list=PhaseList(orix_phase))

In [64]:
from orix import plot
from orix.vector import Vector3d

pg_laue = crystalmap.phases[0].point_group.laue
o_sb = crystalmap['linbo3'].orientations

ipf_key = plot.IPFColorKeyTSL(pg_laue, direction=Vector3d([0, 0, 1]))
rgb_au = ipf_key.orientation2color(o_sb)

In [65]:
pg_laue

Symmetry (12,) -3m
[[ 1.     0.     0.     0.   ]
 [ 0.5    0.     0.     0.866]
 [-0.5    0.     0.     0.866]
 [ 0.     1.     0.     0.   ]
 [ 0.     0.5    0.866  0.   ]
 [ 0.    -0.5    0.866  0.   ]
 [ 1.     0.     0.     0.   ]
 [ 0.5    0.     0.     0.866]
 [-0.5    0.     0.     0.866]
 [ 0.     1.     0.     0.   ]
 [ 0.     0.5    0.866  0.   ]
 [ 0.    -0.5    0.866  0.   ]]

In [52]:
crystalmap['linbo3'].plot(rgb_au)

In [201]:
I = np.eye(3)

ref_ori = orix_Orientation.from_matrix(I)
x_ori = crystalmap[40, 30].orientations

In [215]:
ref_ori.angle_with_outer(x_ori, degrees=True)

array([[79.56828524]])

In [240]:
x_ori

Orientation (1,) mmm
[[0.103  0.7685 0.6087 0.1686]]

In [241]:
x_ori.map_into_symmetry_reduced_zone(verbose=True)

 25%|██▌       | 2/8 [00:00<00:00, 76.01it/s]


Orientation (1,) mmm
[[-0.7685  0.103   0.1686 -0.6087]]

In [299]:
from orix.quaternion import OrientationRegion

Gl, Gr = x_ori._symmetry
symmetry_pairs = list(product(Gl, Gr))

all_ori = []
orientation_region = OrientationRegion.from_symmetry(Gl, Gr)
o_inside = x_ori.__class__.identity(x_ori.shape)
outside = np.ones(x_ori.shape, dtype=bool)
for gl, gr in symmetry_pairs:
    o_transformed = gl * x_ori[outside] * gr
    all_ori.append(gl * x_ori * gr)
    o_inside[outside] = o_transformed
    outside = ~(o_inside < orientation_region)
    #if not np.any(outside):
    #    break'''
o_inside._symmetry = (Gl, Gr)

In [304]:
x_ori.__class__.identity(x_ori.shape)

Orientation (1,) 1
[[1. 0. 0. 0.]]

In [298]:
all_ori[0]

Symmetry (1,) 
[[0.103  0.7685 0.6087 0.1686]]

In [301]:
[ori._symmetry.as_matrix for ori in all_ori]

AttributeError: 'Symmetry' object has no attribute '_symmetry'

In [273]:
outside

array([ True])

In [254]:
Gr

Symmetry (8,) mmm
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  0.  1.]
 [-1.  0.  0.  0.]]

In [252]:
len(symmetry_pairs)

8

In [228]:
from orix.quaternion.orientation import _get_unique_symmetry_elements

symmetry = _get_unique_symmetry_elements(x_ori.symmetry, ref_ori.symmetry)
misorientation = ref_ori * ~x_ori
all_dot_products = Rotation(misorientation).dot_outer(symmetry)

ValueError: setting an array element with a sequence.

In [206]:
ref_ori.unit.dot(crystalmap[40, 30].orientations.unit)

array([0.76846065])

In [205]:
ref_ori.angle_with_outer(crystalmap[40, 30].orientations, degrees=True)

array([[79.56828524]])

In [197]:
crystalmap[40, 30].orientations

Orientation (1,) mmm
[[0.103  0.7685 0.6087 0.1686]]

In [103]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.imshow(rgb_au.reshape(test.map.map_shape))

fig.show()

ValueError: cannot reshape array of size 1572 into shape (81,41)

In [86]:
len(phase_id)

3321

In [88]:
crystalmap.plot('dp')

AttributeError: 'CrystalMap' object has no attribute 'dp'